<img src=".\images\Logo.png" width=150 align="left" /> <img src=".\images\Logo2.jpg" width=450 align="right" />


# <center><font color= #1e8449 > <b>CU04_Optimización de vacunas</font></center>

<font color='steelblue'><b>Citizenlab Data Science Methodology > II - Data Processing Domain </font>
***
> # <font color='steelblue'> <b>05.- Data Collection</font>

<font color='steelblue'>Data Collection is the process to obtain and generate (if required) necessary data to model the problem.</font>

### 04. Secciones censales y zonas básicas de salud

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Settings" data-toc-modified-id="Settings-1"><font color="green">Settings</font></a></span></li><li><span><a href="#Data-Load" data-toc-modified-id="Data-Load-2"><font color="green">Data Load</font></a></span></li><li><span><a href="#ETL-Processes" data-toc-modified-id="ETL-Processes-3">ETL Processes</a></span><ul class="toc-item"><li><span><a href="#Import-data-from:-CSV,-Excel,-Tab,-JSON,-SQL,-and-Parquet-files" data-toc-modified-id="Import-data-from:-CSV,-Excel,-Tab,-JSON,-SQL,-and-Parquet-files-3.1">Import data from: CSV, Excel, Tab, JSON, SQL, and Parquet files</a></span></li></ul></li><li><span><a href="#Synthetic-Data-Generation" data-toc-modified-id="Synthetic-Data-Generation-4">Synthetic Data Generation</a></span></li><li><span><a href="#Fake-Data-Generation" data-toc-modified-id="Fake-Data-Generation-5">Fake Data Generation</a></span></li><li><span><a href="#Open-Data" data-toc-modified-id="Open-Data-6">Open Data</a></span></li><li><span><a href="#Data-Save" data-toc-modified-id="Data-Save-7"><font color="green">Data Save</font></a></span></li><li><span><a href="#Main-Conclusions" data-toc-modified-id="Main-Conclusions-8"><font color=" #2874a6 ">Main Conclusions</font></a></span></li><li><span><a href="#Main-Actions" data-toc-modified-id="Main-Actions-9"><font color=" #2874a6 ">Main Actions</font></a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Acciones-done" data-toc-modified-id="Acciones-done-9.0.1"><font color=" #2874a6 ">Acciones done</font></a></span></li><li><span><a href="#Acctions-to-perform" data-toc-modified-id="Acctions-to-perform-9.0.2"><font color=" #2874a6 ">Acctions to perform</font></a></span></li></ul></li></ul></li></ul></div>

## <font color='green'>Settings</font>

### Packages to use

- {readr} from _tidyverse_ for reading and writing csv files
- {tcltk} for selecting files and paths (if needed)
- {dplyr} for data exploration
- {readxl} for reading excel files
- {stringr} for manipulating strings

In [27]:
library(readr)
library(tcltk)
library(dplyr)
library(readxl)
library(stringr)

### Paths

In [28]:
iPath <- "Data/Input/"
oPath <- "Data/Output/"

## <font color='green'>Data Load</font>

If there are more than one input file, make as many sections as files to import.

#### 1. Correspondencia secciones censales y zonas sanitarias

<font color='tomato'><b> OPCION A:</b> Seleccionar fichero en ventana para mayor comodidad</font> 

Data load using the {tcltk} package. Ucomment the line if not using this option

In [29]:
# file_data <- tcltk::tk_choose.files(multi = FALSE)

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>

Instrucciones
- Los ficheros de entrada del proceso están siempre en Data/Input/.   
- Si hay más de un fichero de entrada, se crean tantos objetos iFile_xx y file_data_xx como ficheros de entrada (xx número correlativo con dos dígitos, rellenar con ceros a la izquierda)

In [30]:
iFile <- "cosalu09.xls"
file_data <- paste0(iPath, iFile)

if(file.exists(file_data)){
    cat("Se leerán datos del archivo: ", file_data)
} else{
    warning("Cuidado: el archivo no existe.")
}


Se leerán datos del archivo:  Data/Input/cosalu09.xls

#### Data file to dataframe

In [31]:
data <- read_excel(file_data)


Estructura de  los datos:

In [32]:
glimpse(data)

Rows: 4,194
Columns: 3
$ `Código zona básica salud`  <chr> "1", "1", "1", "1", "1", "1", "1", "1", "1…
$ `Literal zona básica salud` <chr> "Abrantes", "Abrantes", "Abrantes", "Abran…
$ `Código sección censal`     <chr> "079611157", "079611158", "079611159", "07…


Muestra de datos:

In [33]:
slice_head(data, n = 5)

Código zona básica salud,Literal zona básica salud,Código sección censal
<chr>,<chr>,<chr>
1,Abrantes,079611157
1,Abrantes,079611158
1,Abrantes,079611159
1,Abrantes,079611160
1,Abrantes,079611161


## ETL Processes

### Import data from: CSV, Excel, Tab, JSON, SQL, and Parquet files

Se han importado en el apartado Data Load anterior:

* Correspondencia de zonas básicas de salud y secciones censales de la Comunidad de Madrid

### Extract data

* Filtrar datos de indicadores solo de la Comunidad de Madrid

### Transform data

Para asegurar la trazabilidad, guardar las transformaciones en un objeto diferente con el prefijo `t`

#### Estandarizar tabla de correspondencia secciones

* Nombres de columnas
* Código de zona
* Códigos de municipio - distrito - sección
* Quitar información que no se va a usar

In [34]:
tdata <- data |>
     transmute(id_zona = sprintf("%03s",
                            `Código zona básica salud`),
         nombre_zona = `Literal zona básica salud`,
         id_seccion_censal = `Código sección censal`) |>
    mutate(CMUN = str_sub(id_seccion_censal, end = 3),
           dist = str_sub(id_seccion_censal, 5, 6),
           secc = str_sub(id_seccion_censal, 7, 9)) |>
    select(-id_seccion_censal)

Datos transformados:

In [35]:
glimpse(tdata)

Rows: 4,194
Columns: 5
$ id_zona     <chr> "001", "001", "001", "001", "001", "001", "001", "001", "0…
$ nombre_zona <chr> "Abrantes", "Abrantes", "Abrantes", "Abrantes", "Abrantes"…
$ CMUN        <chr> "079", "079", "079", "079", "079", "079", "079", "079", "0…
$ dist        <chr> "11", "11", "11", "11", "11", "11", "11", "11", "11", "11"…
$ secc        <chr> "157", "158", "159", "160", "161", "162", "163", "165", "1…


In [36]:
tdata |> slice_head(n = 5)

id_zona,nombre_zona,CMUN,dist,secc
<chr>,<chr>,<chr>,<chr>,<chr>
001,Abrantes,079,11,157
001,Abrantes,079,11,158
001,Abrantes,079,11,159
001,Abrantes,079,11,160
001,Abrantes,079,11,161


## Synthetic Data Generation

Estos datos no requieren tareas de este tipo.

## Fake Data Generation

Estos datos no requieren tareas de este tipo.

## Open Data

Los datos se han obtenido de fuentes públicas, ver apartado conclusiones.

## <font color='green'>Data Save</font>

Este proceso, puede copiarse y repetirse en aquellas partes del notebbok que necesiten guardar datos.
Recuerde cambiar la extensión añadida del fichero para diferenciarlas

<font color='tomato'> Identificamos los datos a guardar</font>

In [37]:
data_to_save <- tdata


<font color='tomato'>Estructura de nombre de archivos:</font>

* Código del caso de uso, por ejemplo "CU_04"
* Número del proceso que lo genera, por ejemplo "_05".
* Número de la tarea que lo genera, por ejemplo "_01"
* En caso de generarse varios ficheros en la misma tarea, llevarán _01 _02 ... después
* Nombre: identificativo de "properData", por ejemplo "_zonasgeo"
* Extensión del archivo

Ejemplo: "CU_04_05_01_01_zonasgeo.json, primer fichero que se genera en la tarea 01 del proceso 05 (Data Collection) para el caso de uso 04 (vacunas)

Importante mantener los guiones bajos antes de proceso, tarea, archivo y nombre

### Proceso 05

In [38]:
caso <- "CU_04"
proceso <- '_05'
tarea <- "_04"
archivo <- ""
proper <- "_zonas_secciones"
extension <- ".csv"

<font color='tomato'><b> OPCION A:</b> Uso del paquete "tcltk" para mayor comodidad</font>

* Buscar carpeta, escribir nombre de archivo SIN extensión (se especifica en el código)
* Especificar sufijo2 si es necesario
* Cambiar data por data_xx si es necesario
* Descomentar si se usa esta opción

In [39]:
# file_save <- paste0(caso, proceso, tarea, tcltk::tkgetSaveFile(), proper, extension) 
# path_out <- paste0(oPath, file_save)
# write_csv(data, path_out)

# cat('File saved as: ')
# path_out

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>

- Los ficheros de salida del proceso van siempre a Data/Output/.  

In [40]:
file_save <- paste0(caso, proceso, tarea, archivo, proper, extension) 
path_out <- paste0(oPath, file_save)
write_csv(data_to_save, path_out)

cat('File saved as: ')
path_out

File saved as: 

[1] "Data/Output/CU_04_05_04_zonas_secciones.csv"

#### Copia del fichero a Input

Será usado en otros notebooks

In [44]:
path_in <- paste0(iPath, file_save)
file.copy(path_out, path_in, overwrite = TRUE)

[1] TRUE

## <font color=' #2874a6 '>Main Conclusions</font>

<font color='steelblue'>List and describe the general conclusions of the analysis carried out​.</font>

### Prerequisites

This working code needs the following conditions:

* For using the interactive selection of file, the {tcltk} package must be installed. It is not needed in production.
* The {readr}, {dplyr}, {readxl} and {stringr} packages must be installed. They are part of the _tidyverse_.
* The data paths `Data/Input` and `Data/Output` must exist (relative to the notebook path)

### Configuration Management

This notebook has been tested with the following versions of R and packages. It cannot be assured that later versions work in the same way:
* R 4.2.2
* tcltk 4.2.2
* readr 2.1.3
* dplyr 1.0.10
* readxl 1.4.1
* stringr 1.5.0

### Data structures

#### Objeto `data` (correspondencia secciones censales y zonas de salud)

* Hay 4194 secciones censales asignadas a zonas sanitarias
*  La sección censal se compone de: MMMMDDSSS (código de municipio de 4 dígitos, código de distrito y código de sección censal). Los datos del INE están por código de municipio de 3 dígitos, que son los tres primeros del código de 4 dígitos.
* El código de zona básica viene en numérico, mientras que el archivo de zonas viene en cadena de tres caracteres con ceros a la izquierda.

### Consideraciones para despliegue en piloto

* Datos obtenidos de https://www.madrid.org/iestadis/fijas/clasificaciones/cozousalu.htm.  Archivo `cosalu09.xls`: Zonificación de salud de la Comunidad de Madrid. Se ha descargado a la carpeta `Data/Input`
* Si la información geográfica cambia, se debería actualizar el fichero y volver a ejecutar todos los procesos.


### Consideraciones para despliegue en producción

* Se deben crear los procesos ETL en producción necesarios para que los datos de entrada estén actualizados

## <font color=' #2874a6 '>Main Actions</font>

#### <font color=' #2874a6 '>Acciones done</font>

<font color='steelblue'>Indicate the actions that have been carried out in this process</font>


- Se han transformado los códigos de zonas sanitarias para poder unir las tablas posteriormente
- Se han estandarizado los códigos de municipio, distrito y sección censal

#### <font color=' #2874a6 '>Acctions to perform</font>

<font color='steelblue'>Indicate the actions that must be carried out in subsequent processes</font>

- Se debe asignar a cada sección censal su zona de salud.
- Se deben calcular los indicadores del INE por zona de salud

## <font color='RED'>CODE TO DEPLOY (PILOT)</font>

A continuación se incluirá el código que deba ser llevado a despliegue para producción, dado que se entiende efectúa operaciones necesarias sobre los datos en la ejecución del prototipo

<font color=' red '><b>Description</b></font>

- No hay nada que desplegar en el piloto, ya que estos datos son estáticos o en todo caso cambian con muy poca frecuencia, altamente improbable durante el proyecto.


<font color=' red '><b>CODE</b></font>

In [42]:
# incluir código